# Generative Steps for novel PabB Inhibitors

This notebook contains the steps to use the scorer.py funciton plus chemsampler to sample and generate compounds optimised for PabB binding.

In [5]:
# import necessary libraries
import os
import pandas as pd 
import csv
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit import Chem
from ersilia import ErsiliaModel
from chemsampler import ChemSampler

# set paths for data, results, and source code
DATAPATH = "../data"
RESULTSPATH = "../results"
SOURCEPATH = "../src"

# activate the chemsampler environment (assumed to have been done already)
# (no code is included here as the exact method of activating the environment can vary)

### Initial dataset preparation

The reference_library.csv from https://github.com/ersilia-os/groverfeat/blob/main/data/reference_library.csv was downloaded and addded to the repo. This was filtered based loosely on Lipinski's rule of 5, but with caveats for lead compounds. More info can be found on the wiki page here https://en.wikipedia.org/wiki/Lipinski%27s_rule_of_five.


The aim of producing this dataset of ~200k lead-like compounds is to dock a large number of potential leads into PabB and obtain docking scores. Compounds that dock with a high affinity when compared to the known binders can be used alongside the known binders as a list of compounds to seed chemsampler. We are looking for 50-100 compounds to use as a seed. Chemsampler will then sample and generate ~1 million compounds. These can be filtered, remaining compounds re-docked and further generation can occur.

In [3]:
# Define the paths for the input and output files
smiles_csv = os.path.join(DATAPATH, 'smiles', 'reference_library.csv')
filtered_smiles = os.path.join(DATAPATH, 'smiles', 'testfiltered_std_smiles.csv')

# Read the SMILES strings from the CSV file into a DataFrame
df = pd.read_csv(smiles_csv, skiprows=1, header=None, names=['SMILES']) #can add skiprows=1 is there is a smiles header

# Create an empty list to store the selected SMILES strings
selected = []

# Loop through the SMILES strings and filter by molecular properties
for s in df['SMILES']:
    mol = Chem.MolFromSmiles(s)
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hacceptors = Descriptors.NumHAcceptors(mol)
    hdonors = Descriptors.NumHDonors(mol)
    numrotatablebonds = Descriptors.NumRotatableBonds(mol)
    if 250 <= mw <= 450 and 1 <= logp <= 3 and hacceptors <= 5 and hdonors <= 4 and numrotatablebonds <= 5: 
        selected.append(s)

# Write the selected SMILES strings to a new CSV file
df_selected = pd.DataFrame({'SMILES': selected})
df_selected.to_csv(filtered_smiles, index=False)

In [14]:
len(df["SMILES"]) #how many original smiles?

1999381

In [24]:
len(selected) #how many in our list?

201515

In [45]:
id_list = (range(0, 201515))
len(id_list) # must create a list of 'IDs' for the ID column - this is required for the scorer.py

In [51]:
filtered_selected = os.path.join(DATAPATH, "smiles", "200k_for_chemsampler.csv")

dict = {'ID': id_list, 'SMILES': selected}  
df = pd.DataFrame(dict) 
df.to_csv(filtered_selected, index=False) 

#saves a .csv file with a column for ID and a column for SMILES which will be used to run the first round of docking into PabB.

### Chemsampler

From the docked 200k compounds the top 50 were chosen and seeded to chemsampler. This is to return a long (~1 million?) list of compounds that can be filtered for re-docking.

I also ran Chemsampler using the 2 known binders (abyssomicin C and chorismate), as a test run, to ensure the chemsampler-filtering pipeline works correctly.

Below is the code for the generation of a list of smiles, along with the code to run each sampler individually, and the code to run all of the samplers together. Alterations in the sampler.py file allow for specific samplers to be chosen.

In [6]:
# set the path to the input CSV file
input_csv = os.path.join(DATAPATH, "..", "..", "cstest", "filtered_output.csv")

# set the column index to use (default is 0)
column_index = 0 # this can be changed to 2 if using the outputs of the docking, as it will contain affinity, ID and smiles columns

# read the SMILES strings from the CSV file
smiles_list = []
with open(input_csv, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader) # skip the header row
    for row in csv_reader:
        smiles_list.append(row[column_index])

# store the SMILES strings in a variable
smiles = smiles_list
smiles

['C1C(=C)C2=C3[C@@]4([C@@H1]([C@@H1](O)[C@@H1](O3)[C@@H1](C4)C)/C=C/C(=O)[C@H1]1CC)OC2=O',
 'C=C(O[C@@H1]1C=C(C=C[C@H1]1O)C(=O)O)Br',
 'O1[C@@H1]2[C@H1](O)[C@@H1]3[C@]4(OC(=O)C(=C41)C([C@H1](C)C[C@H1](C)C(Br)/C=C/3)=O)C[C@H1]2C',
 'C1=C/[C@@H1]2[C@H1]([C@@H1]3[C@H1](C)C[C@]24C(=C(C(=O)O4)C(=O)[C@@H1](C[C@H1](C)C/1=O)C)O3)C',
 'C=C(O[C@H1]1[C@H1](O)SC=CC(C(=O)O)=C1)C(=O)O',
 'O=C(C(O[C@@H1]1C=C(C(=O)O)C=C[C@H1]1O)=C)I',
 'C1CC[C@@H1](C(/C=C/[C@@H1]2[C@@H1](O)[C@H1]3OC4=C(C(=O)O[C@@]42C[C@H1]3C)C1)=O)C',
 'OC(=O)C=1C=C[C@@H1](O)[C@@H1](C=1)OC(=C)CS(O)=O',
 'C1=C/[C@@H1]2[C@@H1](O)[C@@H1]3[C@H1](C)C[C@@]24OC(CC([O-1])[C@H1](C)C[C@@H1](C/1=O)C)=C4O3',
 'CC1C[C@H1](C[C@@H1](C)C(C2=C3[C@]4(C[C@H1]([C@@H1]([C@H1](O)[C@H1]41)O3)C)OC2=O)=O)C',
 'O=C(O)C1=CC(OC(C(=O)O)=C(F)F)C(O)C=C1',
 '[C@@H1]1[C@H1]2OC3=C4C(O[C@]3(C[C@H1]2C)[C@@H1]1/C=C/C(=O)[C@H1](C[C@@H1](C)C4=O)C)=O',
 'C=1\\C(C)[C@H1]([C@H1](C)C(=O)C=2C(O[C@]34C=2O[C@@H1]([C@H1](C)C3)[C@@H1]([C@H1]4/C=1)O)=O)CC',
 '[C@H1]12OC=3[C@@]4(OC(=

This is for all samplers, and can tweak in sampler.py

In [ ]:
# running all samplers now with biomodal, moler, and molib turned off

cs = ChemSampler()
resultsALL = cs.sample(smiles, num_samples=1000)
resultsALL

Individual samplers are below

In [ ]:
cs = ChemSampler(samplers_list= ["ChemblSampler"])
resultsCh = cs.sample(smiles, num_samples=1000)
resultsCh


In [ ]:
cs = ChemSampler(samplers_list= ["StonedSampler"])
resultsSt = cs.sample(smiles, num_samples=1000)
resultsSt


In [ ]:
cs = ChemSampler(samplers_list= ["MolerSampler"])
resultsMo = cs.sample(smiles, num_samples=1000)
resultsMo

In [ ]:
cs = ChemSampler(samplers_list= ["PubChemSampler"])
resultsPc = cs.sample(smiles, num_samples=1000)
resultsPc

In [ ]:
cs = ChemSampler(samplers_list= ["SmallWorldSampler"])
resultsSW = cs.sample(smiles, num_samples=1000)
resultsSW

In [ ]:
cs = ChemSampler(samplers_list= ["BimodalSampler"])
resultsBM = cs.sample(smiles, num_samples=1000)
resultsBM

In [ ]:
cs = ChemSampler(samplers_list= ["FasmifraSampler"])
resultsF = cs.sample(smiles, num_samples=1000)
resultsF

In [ ]:
cs = ChemSampler(samplers_list= ["MollibSampler"])
results = cs.sample(smiles, num_samples=1000)
results

### Filtering 

Here, I outline the steps required to filter the output of chemsampler and the docking runs.

Filters to be used:
- MW 250-450
- logP 1-4
- number of rotatable bonds <=5
- number of H bond acceptors <=5
- Number of H bond donors <=5

The above can be done simply in code.

Extra filters to apply, from Ersilia model hub:
- Retrosynthetic accessibility score (cutoff >0.5 - this score is either high or low, Y/N basically)
- Aqueous solubilty (cutoff here -4 to 4, although drugs may be in the -2 to 2 territory. Too soluble and it will be cleared, not soluble enough and it will accumulate)
- Passive permeability (The optimal log10 permeability coefficient (log Papp) for a drug will depend on the specific application and target of the drug. In general, a good log Papp value for a drug should be between -5 and -8 cm/s. Log Papp values in this range indicate that the drug is sufficiently lipophilic to cross cell membranes efficiently, but not so lipophilic that it is unable to dissolve in the watery environment of the body or that it is prone to accumulation in fatty tissues. However, it is important to note that the optimal log Papp value for a specific drug will depend on a variety of factors, including the drug's target tissue or organ, its desired pharmacokinetic profile, and its intended route of administration. Therefore, it is recommended to assess the log Papp of drugs experimentally in vitro and in vivo to determine the optimal range for a specific drug.)


In [10]:
results_list = list(set([x for k,v in resultsALL.items() for x in v])) 
results_list 
# this is turning the results in the resultsALL dictionary into a list

NameError: name 'resultsALL' is not defined

In [ ]:
# Define the path for the output file
filtered_by_desc_output = os.path.join(DATAPATH, '..', '..', 'cstest', 'filtered_by_desc_output.csv')

# Create an empty list to store the filtered SMILES strings
filtered = []

# Loop through the results_list and filter by molecular properties
for s in results_list:
    mol = Chem.MolFromSmiles(s)
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hacceptors = Descriptors.NumHAcceptors(mol)
    hdonors = Descriptors.NumHDonors(mol)
    numrotatablebonds = Descriptors.NumRotatableBonds(mol)
    if 250 <= mw <= 450 and 1 <= logp <= 4 and hacceptors <= 5 and hdonors <= 5 and numrotatablebonds <= 5: 
        filtered.append(s)

# Create a dictionary with the filtered SMILES strings
dict = {'smiles': filtered}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(dict)

# Write the DataFrame to a CSV file
df.to_csv(filtered_by_desc_output, index=False)

All Ersilia models can be run froma a single cell.

In [4]:
filtered_by_desc_ouptut = os.path.join(DATAPATH, "..", "..", "cstest", "filtered_by_desc_output.csv")

In [8]:
# Set paths and filenames
data_path = os.path.join(DATAPATH, '..', '..', 'cstest') # this is only for this run, as the original DATAPATH will be sufficient when running the code later.
filtered_smiles_output = os.path.join(data_path, 'filtered_by_desc_output.csv')
all_filters_output = os.path.join(data_path, 'testall_filters_output.csv')

# Load Ersilia models
ra_model = ErsiliaModel('retrosynthetic-accessibility')
sol_model = ErsiliaModel('eos6oli')
permeability_model = ErsiliaModel('eos2hbd')

# Serve models and run predictions
ra_model.serve()
input_smiles = pd.read_csv(filtered_smiles_output)['smiles']
ra_scores = ra_model.predict(input_smiles, output='pandas')
ra_model.close()

# Filter by RA score
ra_filtered = ra_scores[ra_scores['ra_score'] > 0.5]

sol_model.serve()
solubility = sol_model.predict(ra_filtered['input'].tolist(), output='pandas')
sol_model.close()

# Filter by solubility
sol_filtered = solubility[(solubility['solubility'] > -3) & (solubility['solubility'] < 3)]

permeability_model.serve()
permeability = permeability_model.predict(sol_filtered['input'].tolist(), output='pandas')
permeability_model.close()

# Filter by permeability
permeability_filtered = permeability[(permeability['Log10PermCoeff'] > -8) & (permeability['Log10PermCoeff'] < -4)]

# Drop unnecessary columns, rename input column, and save output to CSV
permeability_filtered.drop(columns=['Log10PermCoeff', 'key'], inplace=True)
permeability_filtered = permeability_filtered.rename(columns={'input': 'smiles'})
permeability_filtered.to_csv(all_filters_output, index=False)

/home/lyndenrooms/miniconda3/envs/chemsampler/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


The below is what was used on the test database..

In [5]:
# now want to run ersilia's models 
all_filters_output = os.path.join(DATAPATH, "..", "..", "cstest", "test.csv")

from ersilia import ErsiliaModel
mdl = ErsiliaModel("retrosynthetic-accessibility")
mdl.serve()
input_smiles = filtered_by_desc_ouptut
df1 = mdl.predict(input_smiles, output="pandas")
mdl.close()

df2 = df1[df1["ra_score"] > 0.01] # this is to be changed to >0.5 for the larger datasets

mdl = ErsiliaModel("eos6oli")
mdl.serve()
df3 = mdl.predict(df2["input"].tolist(), output="pandas")
mdl.close()
df4 = df3[df3["solubility"] > -4]
df4 = df4[df4["solubility"] < 4] # this can be tightened to 3 or 2 for the larger datasets, but is not as important

mdl = ErsiliaModel("eos2hbd")
mdl.serve()
df5 = mdl.predict(df4["input"].tolist(), output="pandas")
mdl.close()
df6 = df5[df5["Log10PermCoeff"] < -4]
df6 = df6[df6["Log10PermCoeff"] > -8]

df6.drop(columns = ["Log10PermCoeff", "key"], inplace=True)
df6 = df6.rename(columns={'input': 'smiles'})
df6.to_csv(all_filters_output, index=False)